Importing all the necessary libraries

In [ ]:
from tensorflow.keras import models, layers, utils
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, Callback, ReduceLROnPlateau
from keras.models import load_model

Linking my google drive account to colab.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Initializing the hyperparameters as needed.

In [ ]:
batch_size = 128
num_classes = 10
l = 6
num_filter = 35
compression = 1.0

Loading all the observations and their corresponding labels.

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes) 

We are using the DenseNet architecture of Convolutional Neural Networks as the our model for training and evaluation. 

In [ ]:
def denseblock(input, num_filter = 12):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        temp = concat
    return temp

In [ ]:
def transition(input, num_filter = 12):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

In [ ]:
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [ ]:
input = layers.Input(shape=(img_height, img_width, channel))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter)
First_Transition = transition(First_Block, num_filter)

Second_Block = denseblock(First_Transition, num_filter)
Second_Transition = transition(Second_Block, num_filter)

Third_Block = denseblock(Second_Transition, num_filter)
Third_Transition = transition(Third_Block, num_filter)

Last_Block = denseblock(Third_Transition,  num_filter)
output = output_layer(Last_Block)

In [ ]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 35)   945         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 35)   140         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 35)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
filepath = '/content/drive/MyDrive/CIFT instance 1'
checkpoint = ModelCheckpoint(filepath=filepath+'/model.hdf5',monitor='val_accuracy',verbose=1, save_best_only=True)
model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])

Training the model and evalution its performence of the test data after a fixed number of epochs.

In [ ]:
datagen = ImageDataGenerator(rotation_range=10,zoom_range=0.1,width_shift_range=0.1,height_shift_range=0.1)
datagen.fit(X_train)
model.fit(datagen.flow(X_train,y_train),batch_size=batch_size,steps_per_epoch=int(len(X_train)/batch_size), epochs=20,validation_data=(X_test, y_test),callbacks=[checkpoint])
model.save_weights(filepath+'/model_epoch20.h5')

Epoch 1/20
390/390 [==============================] - 30s 53ms/step - loss: 1.9513 - accuracy: 0.2888 - val_loss: 2.0713 - val_accuracy: 0.2653

Epoch 00001: val_accuracy improved from -inf to 0.26530, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 2/20
390/390 [==============================] - 19s 49ms/step - loss: 1.5226 - accuracy: 0.4496 - val_loss: 2.2286 - val_accuracy: 0.3441

Epoch 00002: val_accuracy improved from 0.26530 to 0.34410, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 3/20
390/390 [==============================] - 19s 49ms/step - loss: 1.3106 - accuracy: 0.5295 - val_loss: 1.3603 - val_accuracy: 0.5242

Epoch 00003: val_accuracy improved from 0.34410 to 0.52420, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 4/20
390/390 [==============================] - 19s 49ms/step - loss: 1.1400 - accuracy: 0.5890 - val_loss: 1.1706 - val_accuracy: 0.5859

Epoch 00004: val_accuracy improved from 0.524

In [ ]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 4s 12ms/step - loss: 0.3806 - accuracy: 0.8753


[0.3806301951408386, 0.8752999901771545]

In [ ]:
model = load_model('/content/drive/MyDrive/CIFT instance 1/model.hdf5')

In [ ]:
def scheduler(epoch,lr):
  if epoch < 3:
    return lr
  else:
    return lr*0.1

# model.load_weights(filepath+'/pseudo_model_epoch20.h5')
schedule = LearningRateScheduler(scheduler)
model.fit(datagen.flow(X_train,y_train),batch_size=batch_size,steps_per_epoch=int(len(X_train)/batch_size), epochs=20,validation_data=(X_test, y_test),callbacks=[checkpoint, schedule])
model.save_weights(filepath+'/model_epoch40.h5')

Epoch 1/20
390/390 [==============================] - 21s 50ms/step - loss: 0.5397 - accuracy: 0.8119 - val_loss: 0.9317 - val_accuracy: 0.7089

Epoch 00001: val_accuracy did not improve from 0.80250
Epoch 2/20
390/390 [==============================] - 19s 50ms/step - loss: 0.5397 - accuracy: 0.8084 - val_loss: 0.9188 - val_accuracy: 0.7145

Epoch 00002: val_accuracy did not improve from 0.80250
Epoch 3/20
390/390 [==============================] - 19s 49ms/step - loss: 0.5173 - accuracy: 0.8182 - val_loss: 0.7313 - val_accuracy: 0.7631

Epoch 00003: val_accuracy did not improve from 0.80250
Epoch 4/20
390/390 [==============================] - 19s 49ms/step - loss: 0.4262 - accuracy: 0.8551 - val_loss: 0.4888 - val_accuracy: 0.8372

Epoch 00004: val_accuracy improved from 0.80250 to 0.83720, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 5/20
390/390 [==============================] - 19s 50ms/step - loss: 0.3874 - accuracy: 0.8671 - val_loss: 0.4835 - val_ac

For improving the accuracy of the model beyong this, we'll need to add randomness to the data. This can be done by augmenting the images available in the train dataset.

In [ ]:
# model.load_weights(filepath+'/model_epoch40.h5')
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True)
datagen.fit(X_train)
model.optimizer.learning_rate = 1e-4
plateau = ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=3)
model.fit(datagen.flow(X_train, y_train), batch_size=batch_size, steps_per_epoch = int(len(X_train)/batch_size), epochs = 20, validation_data =(X_test, y_test), callbacks=[checkpoint,plateau])
model.save_weights(filepath+'/model_epoch60.h5')

Epoch 1/20
390/390 [==============================] - 20s 48ms/step - loss: 0.4163 - accuracy: 0.8558 - val_loss: 0.4510 - val_accuracy: 0.8485

Epoch 00001: val_accuracy improved from 0.84140 to 0.84850, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 2/20
390/390 [==============================] - 19s 48ms/step - loss: 0.4004 - accuracy: 0.8644 - val_loss: 0.4427 - val_accuracy: 0.8520

Epoch 00002: val_accuracy improved from 0.84850 to 0.85200, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 3/20
390/390 [==============================] - 19s 48ms/step - loss: 0.3942 - accuracy: 0.8627 - val_loss: 0.4282 - val_accuracy: 0.8527

Epoch 00003: val_accuracy improved from 0.85200 to 0.85270, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 4/20
390/390 [==============================] - 19s 48ms/step - loss: 0.3810 - accuracy: 0.8680 - val_loss: 0.4349 - val_accuracy: 0.8543

Epoch 00004: val_accuracy improved from 0.

In [ ]:
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True)
model.optimizer.learning_rate = 1e-4
plateau = ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=2)
datagen.fit(X_train)
model.fit(datagen.flow(X_train, y_train), batch_size=batch_size, steps_per_epoch = int(len(X_train)/batch_size), epochs = 20, validation_data =(X_test, y_test), callbacks=[checkpoint])
model.save_weights(filepath+'/model_epoch80.h5')

Epoch 1/20
390/390 [==============================] - 21s 49ms/step - loss: 0.3256 - accuracy: 0.8897 - val_loss: 0.4083 - val_accuracy: 0.8629

Epoch 00001: val_accuracy did not improve from 0.86920
Epoch 2/20
390/390 [==============================] - 19s 48ms/step - loss: 0.3258 - accuracy: 0.8880 - val_loss: 0.4066 - val_accuracy: 0.8658

Epoch 00002: val_accuracy did not improve from 0.86920
Epoch 3/20
390/390 [==============================] - 19s 48ms/step - loss: 0.3231 - accuracy: 0.8886 - val_loss: 0.3957 - val_accuracy: 0.8689

Epoch 00003: val_accuracy did not improve from 0.86920
Epoch 4/20
390/390 [==============================] - 19s 48ms/step - loss: 0.3268 - accuracy: 0.8849 - val_loss: 0.3918 - val_accuracy: 0.8678

Epoch 00004: val_accuracy did not improve from 0.86920
Epoch 5/20
390/390 [==============================] - 19s 48ms/step - loss: 0.3317 - accuracy: 0.8886 - val_loss: 0.4139 - val_accuracy: 0.8620

Epoch 00005: val_accuracy did not improve from 0.86920


In [ ]:
datagen = ImageDataGenerator(rotation_range = 2, horizontal_flip = True)
model.optimizer.learning_rate = 1e-4
plateau = ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=2)
datagen.fit(X_train)
model.fit(datagen.flow(X_train, y_train), batch_size=batch_size, steps_per_epoch = int(len(X_train)/batch_size), epochs = 20, validation_data =(X_test, y_test), callbacks=[checkpoint,plateau])
model.save_weights(filepath+'/model_epoch100.h5')

Epoch 1/20
390/390 [==============================] - 20s 47ms/step - loss: 0.2613 - accuracy: 0.9101 - val_loss: 0.3603 - val_accuracy: 0.8793

Epoch 00001: val_accuracy improved from 0.87530 to 0.87930, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 2/20
390/390 [==============================] - 18s 47ms/step - loss: 0.2516 - accuracy: 0.9123 - val_loss: 0.3597 - val_accuracy: 0.8774

Epoch 00002: val_accuracy did not improve from 0.87930
Epoch 3/20
390/390 [==============================] - 18s 47ms/step - loss: 0.2391 - accuracy: 0.9183 - val_loss: 0.3549 - val_accuracy: 0.8789

Epoch 00003: val_accuracy did not improve from 0.87930
Epoch 4/20
390/390 [==============================] - 19s 48ms/step - loss: 0.2460 - accuracy: 0.9147 - val_loss: 0.3521 - val_accuracy: 0.8807

Epoch 00004: val_accuracy improved from 0.87930 to 0.88070, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 5/20
390/390 [==============================] - 18s 

In [ ]:
datagen = ImageDataGenerator( horizontal_flip = True, height_shift_range=0.07)
model.optimizer.learning_rate = 1e-4
plateau = ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=2)
datagen.fit(X_train)
model.fit(datagen.flow(X_train, y_train), batch_size=batch_size, steps_per_epoch = 2*int(len(X_train)/batch_size), epochs = 10, validation_data =(X_test, y_test), callbacks=[checkpoint])
model.save_weights(filepath+'/model_epoch130.h5')

Epoch 1/10
780/780 [==============================] - 37s 45ms/step - loss: 0.2596 - accuracy: 0.9100 - val_loss: 0.3482 - val_accuracy: 0.8834

Epoch 00001: val_accuracy improved from 0.88190 to 0.88340, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 2/10
780/780 [==============================] - 35s 44ms/step - loss: 0.2532 - accuracy: 0.9116 - val_loss: 0.3511 - val_accuracy: 0.8824

Epoch 00002: val_accuracy did not improve from 0.88340
Epoch 3/10
780/780 [==============================] - 35s 44ms/step - loss: 0.2485 - accuracy: 0.9159 - val_loss: 0.3519 - val_accuracy: 0.8804

Epoch 00003: val_accuracy did not improve from 0.88340
Epoch 4/10
780/780 [==============================] - 35s 45ms/step - loss: 0.2347 - accuracy: 0.9186 - val_loss: 0.3591 - val_accuracy: 0.8805

Epoch 00004: val_accuracy did not improve from 0.88340
Epoch 5/10
780/780 [==============================] - 35s 44ms/step - loss: 0.2406 - accuracy: 0.9157 - val_loss: 0.3461 - val_ac

In [ ]:
model = load_model('/content/drive/MyDrive/CIFT instance 1/model.hdf5')

In [ ]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 9s 9ms/step - loss: 0.3285 - accuracy: 0.8936


[0.3285086750984192, 0.8935999870300293]

In [ ]:
datagen = ImageDataGenerator( horizontal_flip = True, height_shift_range=0.01)
model.optimizer.learning_rate = 1e-4
plateau = ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=2)
datagen.fit(X_train)
model.fit(datagen.flow(X_train, y_train), batch_size=batch_size, steps_per_epoch = 2*int(len(X_train)/batch_size), epochs = 10, validation_data =(X_test, y_test), callbacks=[checkpoint,plateau])
model.save_weights(filepath+'/model_epoch140.h5')

Epoch 1/10
780/780 [==============================] - 46s 47ms/step - loss: 0.1726 - accuracy: 0.9405 - val_loss: 0.3496 - val_accuracy: 0.8839

Epoch 00001: val_accuracy improved from -inf to 0.88390, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 2/10
780/780 [==============================] - 35s 45ms/step - loss: 0.1674 - accuracy: 0.9432 - val_loss: 0.3586 - val_accuracy: 0.8813

Epoch 00002: val_accuracy did not improve from 0.88390
Epoch 3/10
780/780 [==============================] - 35s 45ms/step - loss: 0.1682 - accuracy: 0.9425 - val_loss: 0.3457 - val_accuracy: 0.8874

Epoch 00003: val_accuracy improved from 0.88390 to 0.88740, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 4/10
780/780 [==============================] - 35s 45ms/step - loss: 0.1616 - accuracy: 0.9456 - val_loss: 0.3397 - val_accuracy: 0.8873

Epoch 00004: val_accuracy did not improve from 0.88740
Epoch 5/10
780/780 [==============================] - 35s 45m

In [ ]:
datagen = ImageDataGenerator( horizontal_flip = True, height_shift_range=0.008)
model.optimizer.learning_rate = 1e-5
plateau = ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=2)
datagen.fit(X_train)
model.fit(datagen.flow(X_train, y_train), batch_size=batch_size, steps_per_epoch = 2*int(len(X_train)/batch_size), epochs = 10, validation_data =(X_test, y_test), callbacks=[checkpoint,plateau])
model.save_weights(filepath+'/model_epoch150.h5')

Epoch 1/10
780/780 [==============================] - 35s 45ms/step - loss: 0.1339 - accuracy: 0.9552 - val_loss: 0.3292 - val_accuracy: 0.8907

Epoch 00001: val_accuracy did not improve from 0.89130
Epoch 2/10
780/780 [==============================] - 35s 45ms/step - loss: 0.1327 - accuracy: 0.9563 - val_loss: 0.3289 - val_accuracy: 0.8910

Epoch 00002: val_accuracy did not improve from 0.89130
Epoch 3/10
780/780 [==============================] - 35s 45ms/step - loss: 0.1277 - accuracy: 0.9591 - val_loss: 0.3295 - val_accuracy: 0.8907

Epoch 00003: val_accuracy did not improve from 0.89130
Epoch 4/10
780/780 [==============================] - 35s 45ms/step - loss: 0.1283 - accuracy: 0.9585 - val_loss: 0.3290 - val_accuracy: 0.8924

Epoch 00004: val_accuracy improved from 0.89130 to 0.89240, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 5/10
780/780 [==============================] - 35s 45ms/step - loss: 0.1315 - accuracy: 0.9554 - val_loss: 0.3301 - val_ac

In [ ]:
datagen = ImageDataGenerator( horizontal_flip = True, height_shift_range=0.005)
model.optimizer.learning_rate = 1e-5
plateau = ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=2)
datagen.fit(X_train)
model.fit(datagen.flow(X_train, y_train), batch_size=batch_size, steps_per_epoch = 2*int(len(X_train)/batch_size), epochs = 10, validation_data =(X_test, y_test), callbacks=[checkpoint,plateau])
model.save_weights(filepath+'/model_epoch160.h5')

Epoch 1/10
780/780 [==============================] - 36s 45ms/step - loss: 0.1275 - accuracy: 0.9588 - val_loss: 0.3285 - val_accuracy: 0.8926

Epoch 00001: val_accuracy improved from 0.89240 to 0.89260, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 2/10
780/780 [==============================] - 35s 45ms/step - loss: 0.1271 - accuracy: 0.9586 - val_loss: 0.3289 - val_accuracy: 0.8927

Epoch 00002: val_accuracy improved from 0.89260 to 0.89270, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 3/10
780/780 [==============================] - 35s 45ms/step - loss: 0.1296 - accuracy: 0.9584 - val_loss: 0.3284 - val_accuracy: 0.8921

Epoch 00003: val_accuracy did not improve from 0.89270
Epoch 4/10
780/780 [==============================] - 35s 45ms/step - loss: 0.1243 - accuracy: 0.9598 - val_loss: 0.3293 - val_accuracy: 0.8926

Epoch 00004: val_accuracy did not improve from 0.89270
Epoch 5/10
780/780 [==============================] - 35s 

In [ ]:
datagen = ImageDataGenerator( horizontal_flip = True, height_shift_range=0.005)
model.optimizer.learning_rate = 1e-5
plateau = ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=2)
datagen.fit(X_train)
model.fit(datagen.flow(X_train, y_train), batch_size=batch_size, steps_per_epoch = 2*int(len(X_train)/batch_size), epochs = 10, validation_data =(X_test, y_test), callbacks=[checkpoint,plateau])
model.save_weights(filepath+'/model_epoch170.h5')

Epoch 1/10
780/780 [==============================] - 27s 31ms/step - loss: 0.1212 - accuracy: 0.9607 - val_loss: 0.3292 - val_accuracy: 0.8934

Epoch 00001: val_accuracy improved from -inf to 0.89340, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 2/10
780/780 [==============================] - 24s 30ms/step - loss: 0.1204 - accuracy: 0.9627 - val_loss: 0.3310 - val_accuracy: 0.8922

Epoch 00002: val_accuracy did not improve from 0.89340
Epoch 3/10
780/780 [==============================] - 23s 30ms/step - loss: 0.1199 - accuracy: 0.9619 - val_loss: 0.3302 - val_accuracy: 0.8925

Epoch 00003: val_accuracy did not improve from 0.89340
Epoch 4/10
780/780 [==============================] - 23s 30ms/step - loss: 0.1167 - accuracy: 0.9629 - val_loss: 0.3303 - val_accuracy: 0.8919

Epoch 00004: val_accuracy did not improve from 0.89340
Epoch 5/10
780/780 [==============================] - 23s 30ms/step - loss: 0.1188 - accuracy: 0.9616 - val_loss: 0.3302 - val_accur

In [ ]:
datagen = ImageDataGenerator( horizontal_flip = True, height_shift_range=0.003)
model.optimizer.learning_rate = 1e-5
plateau = ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=2)
datagen.fit(X_train)
model.fit(datagen.flow(X_train, y_train), batch_size=batch_size, steps_per_epoch = 2*int(len(X_train)/batch_size), epochs = 10, validation_data =(X_test, y_test), callbacks=[checkpoint,plateau])
model.save_weights(filepath+'/model_epoch170.h5')

Epoch 1/10
780/780 [==============================] - 23s 30ms/step - loss: 0.1175 - accuracy: 0.9624 - val_loss: 0.3292 - val_accuracy: 0.8944

Epoch 00001: val_accuracy improved from 0.89390 to 0.89440, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 2/10
780/780 [==============================] - 23s 30ms/step - loss: 0.1187 - accuracy: 0.9615 - val_loss: 0.3292 - val_accuracy: 0.8942

Epoch 00002: val_accuracy did not improve from 0.89440
Epoch 3/10
780/780 [==============================] - 23s 30ms/step - loss: 0.1227 - accuracy: 0.9599 - val_loss: 0.3299 - val_accuracy: 0.8927

Epoch 00003: val_accuracy did not improve from 0.89440
Epoch 4/10
780/780 [==============================] - 23s 30ms/step - loss: 0.1196 - accuracy: 0.9615 - val_loss: 0.3290 - val_accuracy: 0.8934

Epoch 00004: val_accuracy did not improve from 0.89440
Epoch 5/10
780/780 [==============================] - 24s 30ms/step - loss: 0.1174 - accuracy: 0.9628 - val_loss: 0.3295 - val_ac

In [ ]:
datagen = ImageDataGenerator( horizontal_flip = True, height_shift_range=0.001)
model.optimizer.learning_rate = 1e-5
plateau = ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=2)
datagen.fit(X_train)
model.fit(datagen.flow(X_train, y_train), batch_size=batch_size, steps_per_epoch = 2*int(len(X_train)/batch_size), epochs = 10, validation_data =(X_test, y_test), callbacks=[checkpoint,plateau])
model.save_weights(filepath+'/model_epoch180.h5')

Epoch 1/10
780/780 [==============================] - 25s 30ms/step - loss: 0.1210 - accuracy: 0.9601 - val_loss: 0.3293 - val_accuracy: 0.8945

Epoch 00001: val_accuracy improved from 0.89440 to 0.89450, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 2/10
780/780 [==============================] - 23s 30ms/step - loss: 0.1204 - accuracy: 0.9614 - val_loss: 0.3297 - val_accuracy: 0.8931

Epoch 00002: val_accuracy did not improve from 0.89450
Epoch 3/10
780/780 [==============================] - 23s 30ms/step - loss: 0.1218 - accuracy: 0.9607 - val_loss: 0.3294 - val_accuracy: 0.8933

Epoch 00003: val_accuracy did not improve from 0.89450
Epoch 4/10
780/780 [==============================] - 24s 30ms/step - loss: 0.1156 - accuracy: 0.9633 - val_loss: 0.3293 - val_accuracy: 0.8934

Epoch 00004: val_accuracy did not improve from 0.89450
Epoch 5/10
780/780 [==============================] - 23s 30ms/step - loss: 0.1162 - accuracy: 0.9631 - val_loss: 0.3292 - val_ac

In [ ]:
model.load_weights(filepath+'/model_epoch180.h5')

In [ ]:
datagen = ImageDataGenerator( horizontal_flip = True, height_shift_range=0.001)
model.optimizer.learning_rate = 1e-5
plateau = ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=2)
datagen.fit(X_train)
model.fit(datagen.flow(X_train, y_train), batch_size=batch_size, steps_per_epoch = 3*int(len(X_train)/batch_size), epochs = 10, validation_data =(X_test, y_test), callbacks=[checkpoint,plateau])
model.save_weights(filepath+'/model_epoch190.h5')

Epoch 1/10
1170/1170 [==============================] - 36s 29ms/step - loss: 0.1112 - accuracy: 0.9636 - val_loss: 0.3324 - val_accuracy: 0.8940

Epoch 00001: val_accuracy improved from 0.89270 to 0.89400, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 2/10
1170/1170 [==============================] - 34s 29ms/step - loss: 0.1117 - accuracy: 0.9633 - val_loss: 0.3335 - val_accuracy: 0.8934

Epoch 00002: val_accuracy did not improve from 0.89400
Epoch 3/10
1170/1170 [==============================] - 34s 29ms/step - loss: 0.1101 - accuracy: 0.9645 - val_loss: 0.3336 - val_accuracy: 0.8944

Epoch 00003: val_accuracy improved from 0.89400 to 0.89440, saving model to /content/drive/MyDrive/CIFT instance 1/model.hdf5
Epoch 4/10
1170/1170 [==============================] - 34s 29ms/step - loss: 0.1117 - accuracy: 0.9639 - val_loss: 0.3325 - val_accuracy: 0.8938

Epoch 00004: val_accuracy did not improve from 0.89440
Epoch 5/10
1170/1170 [============================

In [ ]:
datagen = ImageDataGenerator(  horizontal_flip=True, width_shift_range=0.001)
model.optimizer.learning_rate = 1e-5
plateau = ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=2)
datagen.fit(X_train)
model.fit(datagen.flow(X_train, y_train), batch_size=batch_size, steps_per_epoch = 3*int(len(X_train)/batch_size), epochs = 10, validation_data =(X_test, y_test), callbacks=[checkpoint,plateau])
model.save_weights(filepath+'/model_epoch200.h5')

Epoch 1/10
1170/1170 [==============================] - 34s 29ms/step - loss: 0.1107 - accuracy: 0.9634 - val_loss: 0.3362 - val_accuracy: 0.8934

Epoch 00001: val_accuracy did not improve from 0.89440
Epoch 2/10
1170/1170 [==============================] - 34s 29ms/step - loss: 0.1071 - accuracy: 0.9656 - val_loss: 0.3380 - val_accuracy: 0.8926

Epoch 00002: val_accuracy did not improve from 0.89440
Epoch 3/10
1170/1170 [==============================] - 34s 29ms/step - loss: 0.1061 - accuracy: 0.9662 - val_loss: 0.3369 - val_accuracy: 0.8928

Epoch 00003: val_accuracy did not improve from 0.89440
Epoch 4/10
1170/1170 [==============================] - 34s 29ms/step - loss: 0.1043 - accuracy: 0.9661 - val_loss: 0.3374 - val_accuracy: 0.8942

Epoch 00004: val_accuracy did not improve from 0.89440
Epoch 5/10
1170/1170 [==============================] - 34s 29ms/step - loss: 0.1037 - accuracy: 0.9662 - val_loss: 0.3370 - val_accuracy: 0.8935

Epoch 00005: val_accuracy did not improve fro

In [ ]:
model = load_model('/content/drive/MyDrive/CIFT instance 1/Copy of model.hdf5')

In [ ]:
#Highest achievable accuracy after multiple attempts
model.evaluate(X_test,y_test)

313/313 [==============================] - 3s 8ms/step - loss: 0.3336 - accuracy: 0.8944


[0.3335798978805542, 0.8944000005722046]

The accuracy of our model did not improve beyond 0.8944. This will be the final state of our model.